In [1]:
import pickle
import pandas as pd
import numpy as np
import time


df = pickle.load( open( "files/df.pickle", "rb" ) )
df['Tweet'] = df['Tweet'].astype(str)
df['Words'] = df['Tweet'].str.count(' ') + 1
df = df[df['Words']>=4]

print('tweets',df.shape[0])
print('Russian',np.sum(df['Category']))

tweets 8488073
Russian 2285628


In [2]:
cv = pickle.load(open('models/countVectorizer.pickle', 'rb'))
sc = pickle.load(open('models/scaler.pickle', 'rb'))
lr = pickle.load(open('models/regression.pickle', 'rb'))

In [3]:
df = df[df['Category']==1].copy()

In [4]:
lst = []
for i, key in enumerate(cv.vocabulary_):
    lst.append((key,lr.coef_[0][cv.vocabulary_[key]]))
    
lst.sort(key = lambda x: x[1], reverse=True)
top = []
for a in lst:
    if a[1] > 0:
        top.append(a)
        
print(len(top),len(cv.vocabulary_))

9811 18272


In [5]:
df['HowRussian'] = lr.predict_proba(sc.transform(cv.transform(df['Processed'])))[:,1]
df = df[df['HowRussian']>=0.9].copy()
df = df.sort_values('HowRussian', ascending=False)
df.head()

,Tweet,Category,Retweet,Processed,Words,HowRussian
897490,Сегодня проголосовали на УИК 738 160 из 446 из...,1,False,JJ NNP NNP VBD CD CD JJ CD JJ NNP NNP CD NNP NN,16,1.000000
787859,Укропы Саханку молотят из 120 и 122-мм. Уже в ...,1,False,JJ NNP NNP VBD CD JJ CD JJ NNP NNP NNP NNP NNP...,24,0.999999
734875,28 июля 988 г -государственная памятная дата Р...,1,False,CD JJ CD JJ NNP NNP NNP NNP NNP VBD CD NN CD N...,19,0.999999
734873,28 июля 988 г -государственная памятная дата Р...,1,False,CD JJ CD JJ NNP NNP NNP NNP NNP VBD CD NN CD N...,19,0.999999
873103,"8 декабря 1991 года, вопреки воле народа, выра...",1,False,CD JJ CD NNP NNP NNP NNP VBD CD NN CD NNP NNP ...,17,0.999999


In [6]:
import string
import re
import nltk

def preprocess_text(series):
     # @somone to @
    lst = []
    for text in series:
         # mark @, # and links
        text = ' '.join(['_at_someone_' if '@' in word else word for word in text.split()])
        text = ' '.join(['_link_' if 'http' in word or 'www.' in word else word for word in text.split()])
        text = ' '.join(['_hashtag_' if '#' in word else word for word in text.split()])
        # remove all other punctuation
        text = ''.join([word.lower() for word in text if word not in string.punctuation.replace('@','').replace('#','').replace('_','')])

        # the real work, pos tagging to reduce Nouns and other things that don't matter
        tokens = []
        for a in nltk.pos_tag(text.split()):
            # keep these as-is for reasons
            if '_link_' in a[0] or '_at_someone_' in a[0] or '_hashtag_' in a[0]:
                tokens.append(a[0])
#             # lemmatize these to reduce parts of speach issues
#             elif 'IN' in a[1] or 'V' in a[1] or 'DT' in a[1] or 'CC' in a[1] or 'MD' in a[1] or 'PDT' in a[1]:
#                 tokens.append(nltk.PorterStemmer().stem(a[0]))
            else:
                tokens.append(a[1])
       
        lst.append(' '.join(tokens))
    return lst 

In [7]:
tweets = ['All men are rapists', 'Both parties are the same.']
lr.predict_proba(sc.transform(cv.transform(preprocess_text(tweets))))

array([[0.30133747, 0.69866253],
       [0.19189906, 0.80810094]])